## Ejemplo 3: Tablas de contingencia

### 1. Objetivos:
    - Aprender a generar tablas de contingencia usando el método `crosstab`
 
---
    
### 2. Desarrollo:

In [1]:
import pandas as pd

In [3]:
df = pd.read_json('../../Datasets/zomato_reviews-clean.json')

df.head()

,has_online_delivery,price_range,currency,name,cuisines,location.address,location.city,user_rating
0,1,3,Rs.,Hauz Khas Social,"Continental, American, Asian, North Indian","9-A & 12, Hauz Khas Village, New Delhi",New Delhi,Very Good
1,0,3,Rs.,Qubitos - The Terrace Cafe,"Thai, European, Mexican, North Indian, Chinese...","C-7, Vishal Enclave, Opposite Metro Pillar 417...",New Delhi,Excellent
2,1,2,Rs.,The Hudson Cafe,"Cafe, Italian, Continental, Chinese","2524, 1st Floor, Hudson Lane, Delhi University...",New Delhi,Very Good
3,0,3,Rs.,Summer House Cafe,"Italian, Continental","1st Floor, DDA Shopping Complex, Aurobindo Pla...",New Delhi,Very Good
4,0,3,Rs.,38 Barracks,"North Indian, Italian, Asian, American","M-38, Outer Circle, Connaught Place, New Delhi",New Delhi,Very Good


Podemos usar el método `crosstab` para generar tablas de contingencia usando dos de nuestras variables categóricas:

In [7]:
pd.crosstab(df['price_range'], df['user_rating'])

user_rating,Average,Excellent,Good,Not rated,Poor,Very Good
price_range,,,,,,
1,12,9,33,1,0,19
2,47,37,116,3,2,132
3,41,84,141,3,1,218
4,19,64,56,19,2,121


Podemos obtener una columna y una fila con los totales añadiendo la bandera `margins` y `margins_name`:

In [9]:
pd.crosstab(df['price_range'], df['user_rating'], margins=True, margins_name='total')

user_rating,Average,Excellent,Good,Not rated,Poor,Very Good,total
price_range,,,,,,,
1,12,9,33,1,0,19,74
2,47,37,116,3,2,132,337
3,41,84,141,3,1,218,488
4,19,64,56,19,2,121,281
total,119,194,346,26,5,490,1180


Como puedes observar, el índice está indicando la primera agrupación de nuestros datos (la columna 'price_range'), mientras que las columnas indican la segunda agrupación (la columna 'user_rating').

---

También podemos añadir una variable categórica más para generar múltiples niveles en las columnas:

In [10]:
pd.crosstab(df['price_range'], [df['has_online_delivery'], df['user_rating']], margins=True, margins_name='total')

has_online_delivery       0                                               1  \
user_rating         Average Excellent Good Not rated Poor Very Good Average   
price_range                                                                   
1                        12         8   32         1    0        15       0   
2                        47        30  104         3    1        99       0   
3                        40        70  132         3    1       183       1   
4                        19        61   52        19    2       113       0   
total                   118       169  320        26    4       410       1   

has_online_delivery                               total  
user_rating         Excellent Good Poor Very Good        
price_range                                              
1                           1    1    0         4    74  
2                           7   12    1        33   337  
3                          14    9    0        35   488  
4                           3    4    0         8   281  
total                      25   26    1        80  1180

### Múltiples niveles en columnas

Es un buen momento para aprender a indexar múltiples niveles en columnas. Recordarás que los multiíndices en filas se indexan de la siguiente forma:

df.loc[(primer_indice, segundo_indice)]

Cuando tenemos múltiples niveles en las columnas, simplemente pasamos como primer valor nuestra indexación por filas, y después una tupla con la indexación por columnas:

In [25]:
crosstab = pd.crosstab(df['price_range'], [df['has_online_delivery'], df['user_rating']])

In [27]:
crosstab

has_online_delivery       0                                               1  \
user_rating         Average Excellent Good Not rated Poor Very Good Average   
price_range                                                                   
1                        12         8   32         1    0        15       0   
2                        47        30  104         3    1        99       0   
3                        40        70  132         3    1       183       1   
4                        19        61   52        19    2       113       0   

has_online_delivery                                
user_rating         Excellent Good Poor Very Good  
price_range                                        
1                           1    1    0         4  
2                           7   12    1        33  
3                          14    9    0        35  
4                           3    4    0         8

In [32]:
crosstab.loc[:, (0)]

user_rating,Average,Excellent,Good,Not rated,Poor,Very Good
price_range,,,,,,
1,12,8,32,1,0,15
2,47,30,104,3,1,99
3,40,70,132,3,1,183
4,19,61,52,19,2,113


In [33]:
crosstab.loc[:, (1, 'Poor')]

price_range
1    0
2    1
3    0
4    0
Name: (1, Poor), dtype: int64

Aquí hemos obtenido la columna 'Poor' del grupo `has_online_delivery == 1`.

---

También otra cosa que podríamos hacer es usar el método `stack`. `stack` lo que hace es tomar una de nuestras columnas y convertirla en índice. Si le pedimos que haga el `stack` en el nivel 0, convertirá el nivel 'has_online_delivery' en índice:

In [34]:
crosstab.stack(level=0)

user_rating                      Average  Excellent  Good  Not rated  Poor  \
price_range has_online_delivery                                              
1           0                         12          8    32        1.0     0   
            1                          0          1     1        NaN     0   
2           0                         47         30   104        3.0     1   
            1                          0          7    12        NaN     1   
3           0                         40         70   132        3.0     1   
            1                          1         14     9        NaN     0   
4           0                         19         61    52       19.0     2   
            1                          0          3     4        NaN     0   

user_rating                      Very Good  
price_range has_online_delivery             
1           0                           15  
            1                            4  
2           0                           99  
            1                           33  
3           0                          183  
            1                           35  
4           0                          113  
            1                            8

In [37]:
stack_level_0 = crosstab.stack(level=0)

stack_level_0.loc[(2)]

user_rating,Average,Excellent,Good,Not rated,Poor,Very Good
has_online_delivery,,,,,,
0,47,30,104,3.0,1,99
1,0,7,12,NaN,1,33


In [38]:
stack_level_0.loc[(2, 1)]

user_rating
Average       0.0
Excellent     7.0
Good         12.0
Not rated     NaN
Poor          1.0
Very Good    33.0
Name: (2, 1), dtype: float64

Si hacemos el stack por el nivel 1, obtenemos lo siguiente:

In [39]:
crosstab.stack(level=1)

has_online_delivery        0     1
price_range user_rating           
1           Average       12   0.0
            Excellent      8   1.0
            Good          32   1.0
            Not rated      1   NaN
            Poor           0   0.0
            Very Good     15   4.0
2           Average       47   0.0
            Excellent     30   7.0
            Good         104  12.0
            Not rated      3   NaN
            Poor           1   1.0
            Very Good     99  33.0
3           Average       40   1.0
            Excellent     70  14.0
            Good         132   9.0
            Not rated      3   NaN
            Poor           1   0.0
            Very Good    183  35.0
4           Average       19   0.0
            Excellent     61   3.0
            Good          52   4.0
            Not rated     19   NaN
            Poor           2   0.0
            Very Good    113   8.0

¡Esto es otro nivel de manipulaciónd de DataFrames!